In [25]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.callbacks import LambdaCallback
import datetime

Все символы в нижний регистр


In [ ]:
filename = "../Датасеты/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

Переводим символы в char


In [27]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 '*': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '3': 11,
 ':': 12,
 ';': 13,
 '?': 14,
 '[': 15,
 ']': 16,
 '_': 17,
 'a': 18,
 'b': 19,
 'c': 20,
 'd': 21,
 'e': 22,
 'f': 23,
 'g': 24,
 'h': 25,
 'i': 26,
 'j': 27,
 'k': 28,
 'l': 29,
 'm': 30,
 'n': 31,
 'o': 32,
 'p': 33,
 'q': 34,
 'r': 35,
 's': 36,
 't': 37,
 'u': 38,
 'v': 39,
 'w': 40,
 'x': 41,
 'y': 42,
 'z': 43,
 'ù': 44,
 '—': 45,
 '‘': 46,
 '’': 47,
 '“': 48,
 '”': 49,
 '\ufeff': 50}

Исследуем данные


In [28]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144678
Total Vocab:  51


Подготавливаем тренировочные данные разделяя всю книгу на последовательности из 100 символов и в качестве ответа выбирая символ следующий за выбранными


In [29]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  144578


нормализуем данные


In [30]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
X.shape
y.shape

(144578, 50)

Создаем архитектуру сети


In [31]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

2024-12-19 20:46:24.834457: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-19 20:46:24.835208: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-19 20:46:24.835868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Так как у нас нет тестовыхх данных, то создаем чекпоинты что бы как то ориентироваться в обучении модели


In [32]:
# define the checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Преобразуем инт в чар


In [33]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

Делаем свой CallBack


In [34]:
def generate_text(epoch, logs):
    if (epoch + 1) % 4 == 0:
        print(f"\n--- Generating text after epoch {epoch + 1} ---")

        start_index = np.random.randint(0, len(dataX) - 1)
        pattern = dataX[start_index]
        print("Seed:")
        print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

        generated_text = ""
        for i in range(200):
            x = np.reshape(pattern, (1, len(pattern), 1))
            x = x / float(n_vocab)
            prediction = model.predict(x, verbose=0)
            index = np.argmax(prediction)
            result = int_to_char[index]
            generated_text += result

            pattern.append(index)
            pattern = pattern[1:len(pattern)]

        print("Generated text:")
        print(generated_text)
        print("\n")


text_generation_callback = LambdaCallback(on_epoch_end=generate_text)

callbacks_list.append(text_generation_callback)

Добавляем TensorBoard


In [5]:
import datetime
log_dir = ("." +
           "logs8lab/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)

callbacks_list.append(tensorboard_callback)

NameError: name 'tf' is not defined

In [36]:
callbacks_list

Обучаем модель


In [37]:
model.fit(X, y, epochs=80, batch_size=128, callbacks=callbacks_list)

Epoch 1/80


2024-12-19 20:46:25.214557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-19 20:46:25.215809: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-19 20:46:25.216636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1130/1130 [==============================] - ETA: 0s - loss: 3.0173
Epoch 1: loss improved from inf to 3.01729, saving model to weights-improvement-01-3.0173.hdf5
1130/1130 [==============================] - 56s 48ms/step - loss: 3.0173
Epoch 2/80
1130/1130 [==============================] - ETA: 0s - loss: 2.8258
Epoch 2: loss improved from 3.01729 to 2.82578, saving model to weights-improvement-02-2.8258.hdf5
1130/1130 [==============================] - 53s 47ms/step - loss: 2.8258
Epoch 3/80
1130/1130 [==============================] - ETA: 0s - loss: 2.7273
Epoch 3: loss improved from 2.82578 to 2.72726, saving model to weights-improvement-03-2.7273.hdf5
1130/1130 [==============================] - 52s 46ms/step - loss: 2.7273
Epoch 4/80
1130/1130 [==============================] - ETA: 0s - loss: 2.6504
Epoch 4: loss improved from 2.72726 to 2.65036, saving model to weights-improvement-04-2.6504.hdf5

--- Generating text after epoch 4 ---
Seed:
" s it went,
“one side will make you

2024-12-19 20:49:58.038434: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-19 20:49:58.039153: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-19 20:49:58.040127: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Generated text:
 wou do woe toue to tee ”ou ”he cat ”ou ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu ”hu 


1130/1130 [==============================] - 62s 55ms/step - loss: 2.6504
Epoch 5/80
1130/1130 [==============================] - ETA: 0s - loss: 2.5825
Epoch 5: loss improved from 2.65036 to 2.58247, saving model to weights-improvement-05-2.5825.hdf5
1130/1130 [==============================] - 54s 48ms/step - loss: 2.5825
Epoch 6/80
1130/1130 [==============================] - ETA: 0s - loss: 2.5207
Epoch 6: loss improved from 2.58247 to 2.52075, saving model to weights-improvement-06-2.5207.hdf5
1130/1130 [==============================] - 53s 47ms/step - loss: 2.5207
Epoch 7/80
1130/1130 [==============================] - ETA: 0s - loss: 2.4669
Epoch 7: loss improved from 2.52075 to 2.46692, saving model to weights-improvement-07-2.4669.hdf5
1130/1130 [==============

Предсказываем


In [54]:
start = np.random.randint(0, len(dataX)-1)
print(start, "data lenght", len(dataX))
pattern = dataX[start]
# got one array
print(pattern)
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    # sys.stdout.write(result)
    print(result, end="")
    pattern.append(index)
    # print(pattern)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

117541 data lenght 144578
[22, 24, 26, 31, 31, 26, 31, 24, 1, 23, 35, 32, 30, 1, 37, 25, 26, 36, 1, 30, 32, 35, 31, 26, 31, 24, 7, 49, 1, 36, 18, 26, 21, 0, 18, 29, 26, 20, 22, 1, 18, 1, 29, 26, 37, 37, 29, 22, 1, 37, 26, 30, 26, 21, 29, 42, 12, 1, 48, 19, 38, 37, 1, 26, 37, 47, 36, 1, 31, 32, 1, 38, 36, 22, 1, 24, 32, 26, 31, 24, 1, 19, 18, 20, 28, 1, 37, 32, 1, 42, 22, 36, 37, 22, 35, 21, 18, 42, 7, 0]
Seed:
" eginning from this morning,” said
alice a little timidly: “but it’s no use going back to yesterday,
 "
bedruiott soey iir soued—.
aod was io alouh simeg an toe fiotens and the carye herlel. and the courd not was ano rired and arlend tha winl she ged not thi was soinging to be toen as oees, and was gorn to her in so shat it was anl aeling the hingeh wfth the tar anwndusd,
the pagb th they har and tha tiol the had shiter and thint shat sie was gows and something ald then she had toiened to thenk the was gown to inne th the oooerssn ana thene was toenened to siink to her onre. 
“h

In [1]:
log_dir

NameError: name 'log_dir' is not defined

In [40]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [51]:
new_log_dir = '/Users/kst_obd/test_folder_for_tesorboard/logs8lab/fit/20241219-204624/train'

In [53]:
%tensorboard --logdir {new_log_dir}

Reusing TensorBoard on port 6006 (pid 33571), started 0:00:05 ago. (Use '!kill 33571' to kill it.)